In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
real = pd.read_excel('data/data.xlsx')
# Меняем названия на более приемлемые:

target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

real = real[real['shop_name'].isin(target_shops)].copy()
real['shop_name'] = real['shop_name'].apply(lambda x: shops_rename[x])
real = real.drop_duplicates(subset=['name'])
real.sample(2)

,shop_name,name,quantity,price,sum
35320,ПЯТЕРОЧКА,"3407480 Сырок РАЭ ванил.твор.16,5% 90г",1.00,32.19,32.19
11384,ПЕРЕКРЕСТОК,5:1537 Мандарины 1кг,0.73,84.90,61.98


In [3]:
real.head()

,shop_name,name,quantity,price,sum
19,ДИКСИ,МИНТАЙ ФИЛЕ Б/К СВ/МОР П/ПАК 8,1.0,199.90,199.90
20,ДИКСИ,БЗМЖ МОЛОКО СТРАНА ВАСИЛЬКИ У/,1.0,49.99,49.99
21,ДИКСИ,СУШКИ ТАРАЛЛИНИ С ЧЕСНОКОМ 180,1.0,29.99,29.99
22,ДИКСИ,ЯЙЦО КУРИНОЕ СТОЛОВОЕ 1КАТЕГОР,1.0,63.99,63.99
23,ДИКСИ,ПЕЧЕНЬЕ ЮБИЛЕЙНОЕ ВИТАМИНИЗИРО,2.0,29.99,59.98


In [4]:
clean = pd.read_csv('clean_data/all_clean.csv')

In [5]:
tmp = pd.read_csv('benchmarks/standard.csv')

In [6]:
def find_weight_or_volume(name: str)-> str:
    """Находит вес или объём товара."""

    pattern1 = r'\d+((\.|\,|x|X|х|Х)?)\d*([а-я]|[А-Я])*$' # ищет вес в конце
    pattern2 = r'\d+\w+' #  ищет вес в середине
    pattern3 = r'(\d+ кг)|(\d+ г)'  # находит `1 кг` или `1 г`
    result = re.search(pattern1, name)
    if result:
        return result.group(0)
    result = re.search(pattern2, name)
    if result:
        return result.group(0)
    result = re.search(pattern3, name)
    return result.group(0) if result else ''

In [7]:
# tmp['Название'].apply(find_weight_or_volume)

In [8]:
tmp['Название'] = tmp['Название'].str.lower()

In [90]:
from product_dict import PRODUCT_DICT

class NormalizeNames:
    def __init__(self, name: pd.Series):
        self.df = pd.DataFrame(name.copy()).rename(columns={'name': 'name_norm'})
        self.df['product_norm'] = None
        self.df['brand_norm'] = None
        
        self.slash_brands = {
            'deluxe/': 'deluxe',
            'nemoloko/': 'nemoloko',
            'huggies/': 'huggies',
            'figaro/': 'figaro',
            'кп/': 'красная птица',
            'кд/': 'каждый день',
            'heinz/' : 'heinz',
            'lay`s/': 'lays',
            'моя цена/': 'моя цена',
            'магнит/': 'магнит',
            'rich/': 'rich',
            '/овощная семейка': 'овощная семейка',
            'dr.oetker/': 'dr. oetker',
            'кр/цена': 'красня цена',
            'д/ваня': 'дядя ваня',
            '/орео': 'oreo',
            'дом.в/дер.': 'домик в деревне',
        }

        self.slash_product = {
            'б/йог': 'биойогурт',
            'сух/завтрак': 'сухой завтрак',
            '/пучок': 'пучок',
            'з/щ': 'зубная щётка',
            'йогурт/смусси': 'йогурт',
            'т/бумага': 'туалетная бумага',
            'укроп/петрушк': 'зелень',
            'б/полотенца': 'бумажные полотенца',
            '/сыр': 'сыр',
            'з/паста': 'зубная паста',
            'т/мыло': 'мыло',
            'ж/мыло': 'жидкое мыло',
            'СУХАРИ/': 'сухари/',
            'бум/салф': 'бумажные полотенца',
            'з/п': 'зубная паста'
            
        }
        self.dot_brands = {
            'д.в д': 'домик в деревне',
            'д.в.д': 'домик в деревне',
            'хл.сп': 'хлебный спас',
            'к.ц': 'красная цена',
            'кр.ц': 'красная цена',
            'кр.цена': 'красная цена',
            'пр!ст': 'просто',
            'п.св': 'первая свежесть',
            'раст': 'растишка',
            'гранд дуэт': 'grand duet',
            'прост.': 'простоквашино',
            'прост ': 'простоквашино',
            'mil gen': 'miller genuine',
            'шарл': 'шарлиз',
            'домашняя кухн': 'домашняя кухня',
            'крикет': 'cricket',
            'домик в дерев': 'домик в деревне',
            'акт.': 'актвия',
            'озор.пчел': 'озорная пчелка',
            'вор.сух': 'воронцовские сухарики',
            'dr.oetker': 'dr.oetker',
            'б.ю.александров': 'б. ю. александров',
            'mr.ricco': 'mr. ricco',
            'dr.korner': 'dr. korner',
            'alp.gold': 'alpen gold',
            'mil.gen.dr' : 'miller',
            'с.прид': 'сады придонья',
            'g.gold': 'gourmet gold',
            'colg.з/п': 'colgate',
            'с.пудовъ': 'с. пудовъ',
            'сибирск.коллекция': 'сибирская коллекция',
            '(данон)': 'danone',
            'lay`': 'lays',
            'кока-кола': 'coca-cola',
            'марс': 'mars',
            'нестле': 'nestle',
            'брест-литовск': 'брест-литовский',
            "lay's": 'lays',
            'хенкель': 'henkel',
            'gl.vil': 'global village',
            'bar': 'barilla',
            'tol': 'tolli',
            'юбил.': 'юбилейное',
            'астор': 'astoria',
            'бар.': 'барни',
            'рот фронт': 'ротфронт',
            'роллт': 'роллтон',
            'верхов.': 'верховье',
            'мист.': 'мистраль',
            'иммун': 'иммунеле',
            'рест.': 'рестория',
            'зд.мен.': 'здоровое меню',
            ' вв ': 'вкусвилл',
            'зелен. линия': 'зеленая линия',
            'комм.': 'коммунарка',
            'волоконовско': 'волоконовское',
            'олвейс': 'always',
            'палмолив': 'palmolive',
            'жатецкий гусь': 'zatecky gus',
            'советские традиц': 'советские традиции',
            'президент': 'president',
            'стародв': 'стародворские',
            'альметте': 'almette',
            'дом в дер': 'домик в деревне',
            'дом дер': 'домик в деревне',
            'дом.в дер.': 'домик в деревне',
            'виола виттр': 'виола виттрока',
            'золот ларец': 'золотой ларец',
            'клинск': 'клинский',
            'добр': 'добрый',
            'рублев': 'рублевский',
            'кремлев': 'кремлевский',
            'гессер': 'gosser',
            'воронц': 'воронцовские',
            'гранола': 'granola',
            'r.sp.': 'ritter sport',
            'sиб.кол.': 'сибирская коллекция',
            'яр.бройл.': 'ярославский бройлер',
            'кузм.': 'кузмино',
            'виола': 'viola',
            'чупа чупс': 'chupa chups',
            'наше солнышк': 'наше солнышко',
            'ролл': 'роллтон',
            'зева': 'zewa',
            'б.ю.алекс.': 'б.ю.александров',
            'мамба': 'mamba',
            'эком.': 'ecomilk',
            'савушк.': 'савушкин хуторок',
            'милка': 'milka',
            'виолетта': 'Violette',
            'марк.пер.': 'маркет перекресток',
            'национ.': 'националь',
            'пр.кав.': 'предгорье кавказа',
            'велк.': 'велком',
            'байс.': 'baisad',
            'ростагроэкс': 'ростагроэкспорт',
            'слав.': 'славянка',
            'варен.': 'варенька',
            'хейнек.': 'heineken',
            'гринфилдголден': 'greenfield',
            'стр.вас.': 'страна васильки',
            'бархатные ручк': 'бархатные ручки',
            'велле': 'velle',
            'терре аллегре': 'terre allegre',
            'зелен.линия': 'зеленая линия',
            'ильина': 'от ильиной',
            'данон': 'danone',
            'юби.': 'юбилейное',
            'больш.кружка': 'большая кружка',
            'рязан.': 'рязаночка',
            'вдох.': 'вдохновение',
            'озби': 'семейка озби',
            'ландлибе': 'landliebe',
            'асен.фер.': 'асеньевская ферма',
            'приос.': 'приосколье',
            'б.ю. алексан': 'б.ю. александров',
            'б.ю.алексан': 'б.ю. александров',
            'б.ю.александров': 'б.ю. александров',
            'б.ю. александров': 'б.ю. александров',
            'азов.кф': 'азовская кф',
            'феликс': 'felix',
            'з.боч': 'золотая бочка',
            'пр.ява': 'принцесса ява',
            'золот.ярлык': 'золотой ярлык',
            'село зел.': 'село зеленое',
            'оral-в': 'оral-b',
            'рог.': 'рогачевъ',
            'золотая симф.': 'золотая симфония',
            'золот стан': 'золотой стандарт',
            'mентос': 'mentos',
            'men.': 'mentos',
            'дан.': 'danone',
            'петр ': 'петр I',
            'ява ': 'ява',
            'лд ': 'LD',
            'люб.': 'любимый',
            'act.': 'activia',
            'сенатор': 'senator',
            'мальборо': 'marlboro',
            'филип моррис': 'philip morris',
            'филлип моррис': 'philip morris',
            'винстон': 'winston',
            'бонд': 'bond',
            'ротманс': 'rothmans',
            'эссе': 'esse',
            'bon': 'bonduelle',
            'магги': 'maggi',
            'чист.лин.': 'чистая линия',
            'рост.': 'ростагроэкспорт',
            'залесск.фермер': 'залесский фермер',
            'kаmis': 'kamis',
            'мистер пропер': 'mr. proper',
            'лейз': 'lays',
            'natur.': 'natural',
            'alw.': 'always',
            'tur.dies.': 'turbo diesel',
            'мдб': 'мясной дом бородина',
            'три короч.': 'три корочки',
            'pur.': 'purina',
            'nest.': 'nestle',
            'bio баланс': 'bio balance',
            'посид.': 'посиделкино',
            'сл.слоб.': 'сладкая слобода',
            'coca-c.': 'coca-cola',
            'хугарден': 'hoegaarden',
            'черног.': 'черноголовка',
            'мерси': 'mersi',
            'чоко пай': 'chocopie',
            'мос.кар.': 'московский картофель',
            'maxib.': 'maxibon',
            'р.хол.': 'русский холод',
            'extr.': 'extreme',
            'свитл.': 'свитлогорье',
            'дом.кух.': 'домашняя кухня',
            'дом кух': 'домашняя кухня',
            'китекат': 'kitekat',
            'контекс': 'contex',
            'лукоил': 'lukoil',
            'орбит': 'orbit',
            'пепси': 'pepsi',
            'провансаль': 'простоквашино',
            'простокваш': 'простоквашино',
            'простоквашино': 'простоквашино',
        }
        self.brands = ['магнит', 'foxlite', 'пятерочка', 'домик в деревне', 'перекресток', 'карусель', 'лента', 'я самая',
                      'billa', 'мягкий знак', 'окей', 'саф-момент', 'рот-фронт', 'му-у', 'm&ms', 'брест-литовский', 'oral-b',
                      'mccormick', 'хлебцы-молодцы', 'coca-cola', 'агро-альянс', 'eco-botanica', 'агуша', 'фрутоняня', 'heinz',
                       'мираторг', 'вкусвилл', 'danone', 'bonduelle', 'hochland', 'greenfield', 'nescafe', 'president', 'бабаевский',
                      'доширак', 'lorenz naturals', 'colgate', 'barilla', 'tolli', 'ролтон', 'юбилейное', 'aura family', 'astoria', 'из углича',
                      'моя цена', 'коммунарка', 'хлебозавод', 'орловский', 'страна васильки', 'дикси', 'первым делом', 'мера вкуса', 'щелковский',
                      'кириешки', 'кузя', 'крестьянское', 'яков давыдов', 'волковская пивоварня', 'дмитровский', 'российский',
                      'завтрак+', 'первак', 'океан', 'маракас', 'круглый год', 'фруто-няня', 'зеленая линия', 'овсяная изюминка', 'наше солнышко',
                      'три корочки', 'тульский', 'особый', 'гиагинский', 'краски лета', 'кока кола', 'маруся', 'фрекен бок', 'индана', 'малиновка', 'любодарово', 'белая долина',
                        'шишкин лес', 'юрма', 'марьин', 'топ шеф', 'красна ягода', 'восточный гость', 'молти', 'праздничная', 'томмолоко', 'орион', 'тараллини',
                       'dirol', 'варвара краса', 'море рядом', 'как раньше', 'цукерман', 'донской табак', 'сиббиттер', 'белевская пастила', 'бифилайф',
                       'степановна', 'отличная цена', 'kotex', 'мясной дом', 'коптильный двор', 'каприз', 'ашан', 'ля фам', 'родные просторы',
                       'даниссимо',
                       
                      ]
        
    @staticmethod
    def remove_all_words_which_contains_number(name, product, brand):
        # Todo: Создать словарь с брендами, у которых есть цифры, например, 365 ДНЕЙ, 7 DAYS
        brands_with_numeric = [
            '365 дней', '7 days', '6 соток', '5 морей', '4 сезона', '7up', '3 glocken', '1toy', '48 копеек',
            'j7', 'got2b', 'хлебозавод 28', '4life','4 life' '36 копеек', 'байкал', '101 зерно', '5 океанов', 'j-7',
            '48коп', 'петр I'
        ]
        
        for br in brands_with_numeric:
            if br in name:
                brand = br
                if brand == '48коп':
                    brand = '48 копеек'
                name = name.replace(br, '')
                break
        name = ' '.join(re.sub(r'\w*\d\w*', '', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_all_words_which_contains_slash(self, name, product, brand):
        for key, value in self.slash_brands.items():
            if key in name:
                brand = value
                break
        
        for key, value in self.slash_product.items():
            if key in name:
                product = value
                break
        name = name.replace('кп/', '').replace('кд/', '')
        name = ' '.join(re.sub(r'\w*/\w*', '', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_punctuation_symbols(self, name, product, brand):
        for key, value in self.dot_brands.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break
        
        tmp = [re.sub(r'([.,!?%:№*\(|\)])', ' ', word).strip() for word in name.split()]
        name = ' '.join(word[:-1] if word.endswith('-') else word for word in tmp)
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_one_and_two_chars(name: str):
#         whitelist = ['с', 'в', 'и', 'из']
        whitelist = ['fa']
        return ' '.join(x.strip() for x in name.split() if len(x) > 2 or x in whitelist)
#         return ' '.join(re.sub(r'\b\w{,2}\b', ' ', word).strip() for word in name.split())
    
    
    def find_brands(self, name, product, brand):
        brands_v2 = {
            'greenf': 'greenfield',
            'hochl': 'hochland',
            'nesc': 'nescafe',
            'ч лин': 'чистая линия',
            'pres': 'president',
            'бабаев': 'бабаевский',
            'дош': 'доширак',
            'бр-лит': 'брест-литовский',
            'lor': 'lorenz naturals',##### ?
            'dan': 'danone',
            'bond': 'bonduelle',
            'colg': 'colgate'
        }
        for br in self.brands:
            if br in name:
                brand = br
                name = name.replace(br, '')
                break
    
        for key, value in brands_v2.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_words_in_blacklist(name: str):
        blacklist = ['бзмж', 'вес', 'для', 'нарезной', 'пшен', 'тенд', 'белый', 'premium', 'декор', 'ранний',
                    'струч', 'короткоплодные', 'венские', 'хлебопек', 'репчатый', 'упак', 'мытая',
                    'дуболистный', 'красный', 'газовая', 'средний', 'слив', 'перс', 'груш', 'классич', 'прод',
                     'раств', 'зерно', 'ябл-бан', 'выгодно', 'красные', 'цветы', 'раст', 'кета', 'косметик',
                    'короткоплодн', 'абр', 'жел', 'шейкер', 'гречн', 'сдоб', 'н-р', '-сл', '[м]', 'вбд',
                     'россия', 'по-домашнему', 'по-корейски', 'киш-миш', 'тд-холдинг', 'дой-пак', 'фрито',
                    'кисло-сладкий', 'рж-пш', 'оквашино', 'нар', 'фас', 'изд', 'охл', 'бел', 'дет', 'паст', 'one',
                    'сливочный', 'зеленью', 'плавл', 'ассорти', 'классическое', 'суп&соус', 'чизбургер', 'ломтик', 'чизбург',
                    'пец-хаас', 'кукурузный', 'ванильный', 'пищевой', 'баварская', 'мульт', 'крупные', 'оливлайн',
                     'черные', 'val', 'плав', 'ломт', 'асс', 'фин', 'избр', 'плавленый', 'финский', 'творожный',
                    'рисунком', 'арт', 'р-р', 'гладкие', 'крас', 'премиум', 'лбп', 'пачка', 'сушеные', 'хлебзернопрод', 'азербайджан', 'розовые',
                    'смо', 'изо', 'ивиа', 'прозр', 'барное', 'светлое', 'темное', 'ультр', 'среднеплодные', 'гладк', 'прополис', 'чесноком', 'карам',
                    'детский', 'яблоко-вишня', 'ишка', 'фасованные', 'детское', 'ржаной', 'украинск', 'оливковый', 'инд', 'окваш', 'моц', 'столовое', 'бокс',
                    'молочный', 'нарез', 'столовая', 'весенние', 'зимой', 'зер', 'зерненый', 'бород', 'белокочанная', 'ранняя', 'домашние', 'морская', 'мелкая', 'рус', 'кривское',
                    'классический', 'ригли', 'вал', 'дрож', 'ябл-черная', 'соев', 'ван', 'москов', 'докторская', 'сух', 'дрожжевое', 
                     'слоеное', 'кокосом', 'зеленый', 'спм', 'вареная', 'выпечки', 'курин', 'пшеничная', 'мытый', 'берканские', 'сладкие',
                    'навелин', 'круглый', 'пинк леди', 'желтый', 'вильямс', 'топл', 'белые', 'спз', 'кдв', 'цыпл-брой', 'сетке',
                     'длинноплодный', 'соленые', 'ский', 'бок', 'тер', 'овские', 'атл', 'маринован', 'гов', 'бородинский', 'рожки', 'пинк', 'хлебоп', 'адриатика',
                    'черн-овес', 'пмм', 'жёлтый', 'дск', 'ветке', 'колотый', 'мга', 'домашний', 'подложка', 'цена', 'детские', 'палермо', 'ромашка', 'фетакса', 'отб',
                    'горький', 'сметанковыймдж', 'комсомольская правда', 'сальчичон', 'гос', 'горшочке', 'красна ягода', 'самокл', 'каймак', 'мягкий', 'кос', 'завар',
                    'шок-ор', 'литовский', 'семейка', 'парижский', 'шашл', 'отборный', 'бакинские', 'отб', 'свежий', 'нарезка', 'форелл', 'стер', 'экон', 'нарезка',
                    'голден', 'дров', 'гладкий', 'больш', 'бриут', 'даниловский', 'франц', 'рислинг', 'тсх', 'тепличные', 'овские', 'стол ']
        return ' '.join(word for word in name.split() if word.strip() not in blacklist)
    
    @staticmethod
    def replace_with_product_dict(name: str):
        return ' '.join(PRODUCT_DICT.get(word.strip(), word) for word in name.split())
    
    @staticmethod
    def replace_most_commen_reductions(name: str):
        reductions = {
            'ср-во': 'средство',
            'филе-кусок': 'филе',
            'нап': 'напиток',
            'крем-мыло': 'мыло',
            'мини-сушки': 'сушки',
            'лук-репка': 'лук',
            'отбел': 'отбеливатель',
            'биотвор': 'биотворог',
            'биойог': 'биойогурт',
            'вин': 'вино',
            'припр': 'приправа',
            'мол': 'молоко',
            'шок': 'шоколад',
            'конф': 'конфеты',
            'жев': 'жевательная резинка',
            'колб': 'колбаса',
            'пакет-майка': 'пакет',
            'твор': 'творог',
            'печен': 'печенье',
            'штруд': 'штрудель',
            'пашт': 'паштет',
            'твор': 'творог',
            'йог': 'йогурт',
            'морож': 'мороженое',
            'салф': 'салфетки',
            'рез': 'резинка',
            'бат': 'батон',
            'майон': 'майонез',
            'марм': 'мармелад',
            'дес': 'десерт',
            'жеврезинка': 'жевательная резинка',
            'шамп': 'шампанское',
            'шокол': 'шоколад',
            'лим кислота': 'лимонная кислота',
            'том.паста': 'томатная паста',
            'ваф': 'вафли',
            'пельм': 'пельмени',
            'спагет': 'спагетти',
            'мюс': 'мюсли',
            'кор': 'коре',
            'наполн': 'наполнитель',
            'пирож': 'пирожное',
            'макар': 'макароны',
            'порош': 'порошок',
            'tоматы': 'томаты',
            'сред': 'средство',
            'свечи-мини': 'свечи',
            'проклад': 'прокладки',
            'хлоп': 'хлопья',
            'шам': 'шампунь',
            'нект': 'нектар',
            'зп': 'зубная паста',
            'сухар': 'сухари',
            'сосис': 'сосиски',
            'приправ': 'приправа',
            'котл': 'котлета',
            'прокл': 'прокладки',
            'салфет': 'салфетки',
            'мармел': 'мармелад',
            'печ': 'печенье',
            'напит': 'напиток',
            'багета': 'багет',
            'горош': 'горошек',
            'мак': 'макароны',
            'семеч': 'семечки',
            'кренд': 'крендели',
            'пасх': 'пасха',
            'зажиг': 'зажигалка',
            'батонч': 'батончик',
            'сырчечилспаг': 'сыр',
            'ароматиз': 'ароматизатор',
            'наб': 'набор',
            'антиперсп': 'антиперспант',
            'бальз-опол': 'бальзам',
            'бзмжсыр': 'сыр',
            'био-йогу': 'биойогурт',
            'био-йогурт': 'биойогурт',
            'биотв': 'биотворог',
            'бумаг': 'бумага',
            'вермиш': 'вермишель',
            'гель-эксп': 'гель',
            'зажигалок': 'зажигалка',
            'кетч': 'кетчуп',
            'кнофеты': 'конфеты',
            'крем-мыл': 'мыло',
            'кэфир': 'кефир',
            'лосьон-тон': 'лосьон',
            'макарон': 'макароны',
            'манг': 'манго',
            'морожен': 'мороженое',
            'пирожн': 'пирожное',
            'подг': 'подгузники',
            'презерв': 'презервативы',
            'сосиск': 'сосиски',
            'тамп': 'тампоны',
            'наб': 'набор',
            'биойог': 'биойогурт',
            'кетч': 'кетчуп'
        }
        return ' '.join(reductions.get(word.strip(), word) for word in name.split())
        
    def find_scratch_card(self, name, product, brand):
        if 'скретч-карта' in name:
            return pd.Series(['', 'скретч-карта', 'перекресток'])
        return pd.Series([name, product, brand])
    
    @staticmethod
    def find_product(name, product):
        if not product:
            if len(name.split()) == 1:
                if name == 'мол':
                    return pd.Series(['', 'молоко'])
                product = name
                return pd.Series(['', product])
            return pd.Series([name, None]) 
        return pd.Series([name, product])

    @staticmethod
    def find_and_remove_all_english_words(name, brand):
        eng_brands = re.findall(r'\b([a-z]+)\b', name)
        if eng_brands and not brand:
            result = []
            for word in eng_brands:
                name = name.replace(word, '').replace('  ', ' ').strip()
                result.append(word)

            brand = ' '.join(result)
            if 'epica' in brand:
                brand = 'epica'
            return pd.Series([name, brand])

        # If we have brand, we should delete english words in product name
        name = re.sub(r'\b([a-z]+)\b', '', name).replace('  ', ' ').strip()
        return pd.Series([name, brand])
    
    def normalize(self):
        self.df['name_norm'] = self.df['name_norm'].str.lower()
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_number(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_slash(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_with_product_dict)
        
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_punctuation_symbols(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_brands(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_one_and_two_chars)
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_words_in_blacklist)
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_most_commen_reductions)
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_scratch_card(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_and_remove_all_english_words(x['name_norm'], x['brand_norm']), axis=1
        )
        self.df[['name_norm', 'product_norm']] = self.df.apply(
            lambda x: self.find_product(x['name_norm'], x['product_norm']), axis=1
        )
        return self.df

In [91]:
%%time
n = NormalizeNames(real['name'])
tmp = n.normalize()
# Wall time: 1min 24s

CPU times: user 1min 20s, sys: 609 ms, total: 1min 21s
Wall time: 1min 22s


In [92]:
df = pd.concat([real, tmp], axis=1)[['shop_name', 'name', 'name_norm', 'product_norm', 'brand_norm']]

In [93]:
df.brand_norm.dropna().shape # 12912

(12912,)

In [94]:
df.product_norm.dropna().shape # 7589

(7589,)

In [14]:
from typing import Optional
from itertools import combinations

class FinderInEtalon:
    def __init__(self, df: pd.DataFrame, path_to_brands: str = 'clean_data/rus_brands.csv', path_to_products: str = 'clean_data/products.csv'):
        self.rus_brands = pd.read_csv(path_to_brands)['Бренд'].values
        self.products = pd.read_csv(path_to_products)
        self.columns = ['name_norm', 'product_norm', 'brand_norm']
        self.df = df[self.columns].copy()
        
    def find_brands(self, name: str, brand: Optional[str] = None) -> pd.Series:
        if name and not brand:
            names = set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split())
            for rus_brand in self.rus_brands:
                if rus_brand in names:
                    name = name.replace(rus_brand, '').replace('  ', ' ').strip()
                    return pd.Series([name, rus_brand])
        return pd.Series([name, brand])

    @staticmethod
    def find_one_product(name, product):
        if not product and len(name.split()) == 1:
            product = name
            return pd.Series(['', product])
        return pd.Series([name, product])
    
    
    def find_all(self):
        self.df[['name_norm', 'brand_norm']] = self.df.apply(lambda x: self.find_brands(x['name_norm'], x['brand_norm']), axis=1)
        self.df[['name_norm', 'product_norm']] = self.df.apply(lambda x: self.find_one_product(x['name_norm'], x['product_norm']), axis=1)
        
        return self.df

In [15]:
%%time
finder = FinderInEtalon(df)
res = finder.find_all()
# 21.8 s

CPU times: user 21.8 s, sys: 62.5 ms, total: 21.9 s
Wall time: 22.1 s


In [16]:
res.brand_norm.dropna().shape # 17130

(17130,)

In [17]:
from itertools import combinations
from typing import List

products = pd.read_csv('clean_data/products.csv')

def remove_duplicate_word(arr: List[str]) -> List[str]:
    """
    Remove duplicate words from products:
    ['вода', 'вода питьевая'] --> ['вода питьевая']
    """
    
    if max([len(x.split()) for x in arr]) > 1:
        arr = sorted(arr, key=lambda x: len(x.split()))
        one_words = []
        for product in arr.copy():
            if len(product.split()) == 1:
                one_words.append(product)
            else:
                for word in one_words:
                    if word in product:
                        arr.remove(word)
    return arr

def find_product(name: str, product: str):
    if name and not product:
        names = pd.DataFrame(set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split()), columns=['Продукт'])
        merge = products.merge(names)
        if len(merge):
            product = ', '.join(remove_duplicate_word(merge['Продукт'].values))
            category = merge['Категория'].value_counts().index[0]
            return pd.Series([name, product, category])
    return pd.Series([name, product, None])

In [18]:
%%time
res[['name_norm', 'product_norm', 'cat_norm']] = res.apply(lambda x: find_product(x['name_norm'], x['product_norm']), axis=1)
# Wall time: 1min 37s


CPU times: user 1min 34s, sys: 297 ms, total: 1min 35s
Wall time: 1min 37s


In [19]:
res = pd.concat([df['name'], res], axis=1)

In [20]:
res.product_norm.dropna().shape # 9343 -> 27570 -> 28930 -> 29617 -> 28684 -> 28744 -> 29316 -> 29455 -> 29496

(29496,)

In [44]:
l = []

def find_category(product: str, cat: str) -> str:
    if product and not cat:
        tmp = products[products['Продукт'] == product]
        if not len(tmp):
            l.append(product)
        else:
            return tmp['Категория'].values[0]
    return None

In [45]:
kek = res.apply(lambda x: find_category(x['product_norm'], x['cat_norm']), axis=1)

In [ ]:
{
    'шамп/бал': '',
    'с/порошок': '',
    'ш/р': '',
}

In [57]:
'Ш/Р'.lower()

'ш/р'

In [ ]:
{
    'китекат': 'kitekat',
    'контекс': 'contex',
    'лукоил': 'lukoil',
    'орбит': 'orbit',
    'пепси': 'pepsi',
    'провансаль': '',
    'простокваш': '',
    'простоквашино': '',
    '': '',
    '': '',
    '': '',
    '': '',
    '': '',
    '': '',
    '': '',
    
    
}

In [61]:
res[res['product_norm'] == '-кабель']

,name,name_norm,product_norm,brand_norm,cat_norm
20177,1: 4029336 USB-кабель TYC-2 100см,,-кабель,usb tyc,None


Красота, гигиена, бытовая химия
Хлеб, сладости, снеки
Молоко, сыр, яйца
Соусы, орехи, консервы
Макароны, крупы, специи
Другое
Воды, соки, напитки
Рыба, икра
Чай, кофе, сахар
Замороженные продукты
Птица, мясо, деликатесы
Посуда
Овощи, фрукты, ягоды
Товары для мам и детей
Товары для дома и дачи
Подборки и готовые блюда
Зоотовары
Бытовая техника
Алкоголь
Дача и гриль

In [70]:
from IPython.display import clear_output


black_list = []
white_list = {}

for i, product in enumerate(np.unique(l)):
    print(product, f'{i+1} from {len(np.unique(l))}', sep='')
    in_ = input()
    if in_ == '':
        black_list.append(product)
    else:
        white_list[product] = in_
    clear_output()

In [ ]:
{
    'антиперсп': 'антиперспант',
    'бальз-опол': 'бальзам',
    'бзмжсыр': 'сыр',
    'био-йогу': 'био-йогурт',
    'био-йогурт': 'био-йогурт',
    'биотв': 'био-творог',
    'бумаг': 'бумага',
    'вермиш': 'вермишель',
    'гель-эксп': 'гель',
    'зажигалок': 'зажигалка',
    'кетч': 'кетчуп',
    'кнофеты': 'конфеты',
    'крем-мыл': 'мыло',
    'кэфир': 'кефир',
    'лосьон-тон': 'лосьон',
    'макарон': 'макароны',
    'манг': 'манго',
    'морожен': 'мороженое',
    'пирожн': 'пирожное',
    'подг': 'подгузники',
    'презерв': 'презервативы',
    'сосиск': 'сосиски',
    'тамп': 'тампоны',
    'наб': 'набор'
}

In [336]:
res[(res['name_norm'].str.find('наб ') != -1)&(res['product_norm'].apply(lambda x: not bool(x)))]

,name,name_norm,product_norm,brand_norm,cat_norm
12883,16:3942069 Наб.мини-шок.RITTER SPORT 116г,наб мини-шок,None,ritter sport,None
12956,5: 3178908 ДОМ.КУХ.Наб.ПРОСТО!БЫСТРО!пас,дом кух наб быстро пас,None,простоквашино,None
14082,15*: 3384560 ДОМ.КУХ.Наб.форм бум.д/кули,наб форм бум,None,домашняя кухня,None
14083,16*: 3384560 ДОМ.КУХ.Наб.форм бум.д/кули,наб форм бум,None,домашняя кухня,None
15283,8*: 4019592 ДОМ.КУХ.Наб.ПЕР.КР.см.д/окр.,наб пер,None,домашняя кухня,None
15304,29*: 4019591 ДОМ.КУХ.Наб.см.КР.В ТАБ.д/о,наб таб,None,домашняя кухня,None
30672,*4027521 УВЕЛКА Наб.ПАС.В СОУСЕ ПЕС.340г,наб пас соусе пес,None,увелка,None
30674,*4027522 УВЕЛКА Наб.ПАС.АЛ.В СЛ-Ч.С.340г,наб пас сл-ч,None,увелка,None
34099,*4026121 Наб.накл.д/дек.яиц ST4500 20эл.,наб накл яиц,None,None,None
34578,*4027515 УВЕЛКА Наб.Б.В С-Ч.С.ПО-ИТ.260г,наб с-ч по-ит,None,увелка,None


TODO:
1. Разделить по '-'
2. найти категории к оставшимся товарам

провансаль - добавить

}

In [95]:
black_list

['+',
 '+темн',
 '-',
 '-бана',
 '-огор',
 '-пятнов',
 '-ржан',
 '-це',
 'bаф',
 'cofullотбал',
 'hсельдьпо',
 'kotanyiприправаитал',
 'maх',
 'proхвост',
 'villфасоль',
 'абсолют',
 'авт',
 'агар',
 'агар-агар',
 'аджапсандал',
 'азерчай',
 'аистенок',
 'айс',
 'амка',
 'анан-манго-марк',
 'антиб',
 'антидождь',
 'антип-нт',
 'антиперсп',
 'антиперсп-кар',
 'антисеп',
 'апел-маракуйя',
 'апельс-марак',
 'аппетитная',
 'ацидобифили',
 'бал',
 'бальз-опол',
 'бамб',
 'бамия',
 'банк',
 'батmax+',
 'батар',
 'батард',
 'батат',
 'батбалансbite',
 'бежицкие',
 'без',
 'беконпо-испански',
 'белая',
 'белизнагель',
 'белья',
 'бенкизер',
 'бер',
 'бефстрог',
 'бефстроганов',
 'бзмжсыр',
 'био-йогу',
 'био-йогурт',
 'биопрод',
 'биопродукткефирный',
 'биотв',
 'бого',
 'бородина',
 'бразильские',
 'брауншвейгск',
 'бриз',
 'бужен',
 'булгур',
 'бульмени',
 'бумаг',
 'бутерхлеб',
 'бюветъ',
 'вантуз',
 'вермиш',
 'винтурдеманборд',
 'виш-гран',
 'вод',
 'водкакизляркатрад',
 'возр',
 'вологж'

In [337]:
res[res['product_norm'].apply(lambda x: not bool(x))].sample(10)

,name,name_norm,product_norm,brand_norm,cat_norm
46115,ОЧИСТ ВЕЛЮР И КОВР,очист велюр ковр,None,None,None
29341,3471063 ХЛЕБОК.Батоны НАРЕЗ.в/с нар380г,хлебок батоны,None,None,None
44037,КД/ПРОСТЫНЯ ЕВРО ПЕР,ыня евро пер,None,простоквашино,None
43013,БИСК КЛ/ВАН БАРНИ,бисквит барни,None,None,None
49372,ПЕЧ ШОКОЛАДНОЕ САХ,шоколадное сах,None,None,None
5792,"Биокефир 2,5% 0,5кг (Пискарёвский МЗ):10",биокефир пискарёвский,None,None,None
43878,??? ?????????? 900?,,None,None,None
44385,РОДНЫЕ ПРОСТОРЫ BАФ,родные оры bаф,None,простоквашино,None
51160,РОЗА Ч/Г ЛИМБО (20ШТ,роза лимбо,None,None,None
43539,"ЛИПЕЦ.МИНЕР.ГАЗ.1,5Л",липец минер газ,None,None,None


In [338]:
'РОДНЫЕ ПРОСТОРЫ'.lower()

'родные просторы'

In [281]:
C.most_common(40)

[('мак', 46),
 ('мор', 42),
 ('дом', 32),
 ('сол', 30),
 ('наб', 28),
 ('пасх', 28),
 ('микс', 26),
 ('ядра', 23),
 ('кислота', 22),
 ('сем', 22),
 ('жен', 21),
 ('бум', 21),
 ('пит', 20),
 ('кар', 19),
 ('суш', 18),
 ('масса', 17),
 ('чер', 17),
 ('нач', 17),
 ('конд', 16),
 ('теста', 15),
 ('коп', 15),
 ('лим', 15),
 ('шар', 14),
 ('куриные', 14),
 ('нат', 14),
 ('пищ', 14),
 ('карт', 14),
 ('кухня', 14),
 ('мясом', 13),
 ('жар', 13),
 ('посыпки', 12),
 ('мякоть', 12),
 ('шарики', 12),
 ('сах', 12),
 ('вар', 12),
 ('смс', 12),
 ('туал', 12),
 ('красит', 12),
 ('изделия', 11),
 ('лимонная', 11)]

In [275]:
res[res['product_norm'].apply(lambda x: not bool(x))].sample(10)

,name,name_norm,product_norm,brand_norm,cat_norm
49478,РЕДЬКА ЧЕРНАЯ,редька черная,None,None,None
45885,ВЕНСКИЙ КОНВ.ЧЕР.70Г,венский конв чер,None,None,None
16802,3*: 32340 ДЯДЯ ВАНЯ Горош.зелен.ж/б 400,горош зелен,None,дядя ваня,None
45924,КП/ДЕТСКАЯ ВЛАЖНАЯ,детская влажная,None,красная птица,None
62978,Треугольник с кусочками яблок 110г,треугольник кусочками яблок,None,None,None
16311,*3421780 ГОР.ШТ.Изд.ЧЕБУП.КУР.ПО-ИТ.250г,гор чебуп кур по-ит,None,None,None
25559,3488543 П.СВ.Бифшт.ИЗОБИЛИЕ бек.600г,бифшт изобилие бек,None,первая свежесть,None
7100,Эскимо Энергия слив. 70г(Саратов-Хол,эскимо энергия саратов-хол,None,None,None
13156,3361848 LIBERH.Освежит.ПОСЛЕ ДОЖДЯ300мл,освежит после,None,liberh,None
43831,ШПРИЦ ОДНОРАЗ 5 МЛ,шприц однораз,None,None,None


In [205]:
'сахар-песок' in products['Продукт'].values

False

In [196]:
br = pd.read_csv('clean_data/rus_brands.csv') #926
br[br['Бренд'] != 'еко'].to_csv('clean_data/rus_brands.csv', index=False)

In [143]:
pd.concat([br, pd.DataFrame(['alw'], columns=['Бренд'])]).to_csv('clean_data/rus_brands.csv', index=False)

In [82]:
q = pd.read_csv('clean_data/products.csv')

In [169]:
q[q['Продукт'] == 'яблоки']

,Продукт,Категория


In [151]:
q.drop(index=461).to_csv('clean_data/products.csv', index=False)

In [112]:
q.loc[446]['Категория'] = 'Овощи, фрукты, ягоды'

In [137]:
q.loc[950]

Продукт        изд "мозаика-синтез"
Категория    Товары для мам и детей
Name: 950, dtype: object

In [115]:
q.to_csv('clean_data/products.csv', index=False)

In [284]:
'макароны' in q['Продукт'].values

True

In [ ]:
'открытка', 'поджарка', 'белок', 'плитка', 'окорочка', 'гвоздика', 'горошек', 'редька', 'рассол', 'пасха', 'кислота лимонная'

In [38]:
q['Категория'].unique()

array(['Красота, гигиена, бытовая химия', 'Хлеб, сладости, снеки',
       'Молоко, сыр, яйца', 'Соусы, орехи, консервы',
       'Макароны, крупы, специи', 'Другое', 'Воды, соки, напитки',
       'Рыба, икра', 'Чай, кофе, сахар', 'Замороженные продукты',
       'Птица, мясо, деликатесы', 'Посуда', 'Овощи, фрукты, ягоды',
       'Товары для мам и детей', 'Товары для дома и дачи',
       'Подборки и готовые блюда', 'Зоотовары', 'Бытовая техника',
       'Алкоголь', 'Дача и гриль'], dtype=object)

In [78]:
tmp = pd.DataFrame.from_dict(white_list, orient='index').reset_index().rename(columns={'index': 'Продукт', 0: 'Категория'})

In [ ]:
'дыня', 'краситель', 'cалат', 'коржик'

In [83]:
tmp = pd.DataFrame(
    [
        ['дыня', 'Овощи, фрукты, ягоды'],
        ['краситель', 'Другое'],
        ['cалат', 'Овощи, фрукты, ягоды'],
        ['коржик', 'Хлеб, сладости, снеки'],
    ], columns=['Продукт', 'Категория']
)

In [84]:
pd.concat([q, tmp]).to_csv('clean_data/products.csv', index=False)

In [85]:
q.shape

(1265, 2)

In [88]:
q.drop_duplicates(subset='Продукт').to_csv('clean_data/products.csv', index=False)

In [279]:
from collections import Counter

C = Counter()
def lol(name):
    global C
    for i in name.split():
        C[i]+=1



In [280]:
res[res['product_norm'].apply(lambda x: not bool(x))]['name_norm'].apply(lol);